In [1]:
"""
이미지를 색상에 맞춰 폴더로 분류 및 색상의 비율

파라미터 설명

each_vienna_max = 비엔나코드당 추출할 이미지의 갯수
threshold = 가중치
match_category = 특정 색에 대한 추출 (현재는 모든 색에 대한 추출로 변경)

"""
from config import lower_color_dic, upper_color_dic,src_dir, color_data_path


def all_copy_with_sub_folder(each_vienna_max=1,
                             threshold=10,
                             match_category=None,
                             ):
    # 갈색  =  108
    # 검정  =  2942
    # 남색1  =  122
    # 남색2  =  53
    # 노랑1  =  24
    # 노랑2  =  260
    # 보라1  =  42
    # 보라2  =  3
    # 보라3  =  22
    # 분홍  =  151
    # 빨강  =  1875
    # 연두  =  83
    # 자주  =  88
    # 주황  =  947
    # 청록  =  99
    # 초록  =  95
    # 파랑1  =  12
    # 파랑2  =  57
    # 파랑3  =  892
    # 파랑4  =  120
    # 회색  =  505

    def return_color(select_color, is_lower=True):
        if is_lower:
            return lower_color_dic[select_color]

        else:
            return upper_color_dic[select_color]

    import numpy as np
    import os
    import shutil
    import cv2


    category_list = list(upper_color_dic.keys())
    vienna_code_list = os.listdir(src_dir)

    # Loop through the files and copy them to the destination directory if they are choosen color
    for category_folder in category_list:

        #해당 카테고리만 카피
        if category_folder != match_category:
            continue

        print(category_folder)
        dst_dir = f'{color_data_path}/{category_folder}/'

        for vienna_code_folder in vienna_code_list:

            #2번이후로 리스트 생략
            #if vienna_code_folder[:1] == '2':
            #break 

            if vienna_code_folder == ".DS_Store":
                break
            image_list = os.listdir(os.path.join(src_dir, vienna_code_folder))

            #count each vienna code
            COUNT_VIENNA = 0

            for image_path in image_list:

                # Load the image
                img = cv2.imdecode(np.fromfile(os.path.join(src_dir, vienna_code_folder, image_path), dtype=np.uint8),
                                   cv2.IMREAD_UNCHANGED)
                if img is not None:

                    if category_folder == '회색':

                        try:

                            # Create a boolean mask with the same shape as the image, initialized to False
                            mask = np.zeros(img.shape[:2], dtype=bool)

                            # Find pixels where R, G, and B values are equal (gray pixels) and set them to True in the mask
                            mask[(img[:, :, 0] == img[:, :, 1]) & (img[:, :, 0] == img[:, :, 2])] = True

                            # Create a copy of the original image
                            mask_img = img.copy()

                            # Set the pixels in the masked image that are not gray to zero
                            mask_img[~mask] = 0

                            # Set the lower and upper bounds for gray colors
                            lower_color = np.array(return_color(category_folder))
                            upper_color = np.array(return_color(category_folder, False))

                            # Create a mask for gray pixels in the range
                            mask_range = cv2.inRange(mask_img, lower_color, upper_color)

                            # Count the number of white pixels in the mask (i.e., gray pixels within range)
                            pixels_in_range = cv2.countNonZero(mask_range)

                            # Calculate the total number of pixels in the image
                            total_pixels = img.shape[0] * img.shape[1]

                            percentage = (pixels_in_range / total_pixels) * 100

                            if percentage > threshold:

                                if not os.path.exists(dst_dir):
                                    os.makedirs(dst_dir)

                                shutil.copy(os.path.join(src_dir, vienna_code_folder, image_path), dst_dir)

                                COUNT_VIENNA += 1
                                #escape if count match max 
                                if COUNT_VIENNA >= each_vienna_max:
                                    print(vienna_code_folder, COUNT_VIENNA)
                                    break

                            continue
                        except Exception:
                            continue

                    # Convert the image to RGB color space
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    lower_color = np.array(return_color(category_folder))
                    upper_color = np.array(return_color(category_folder, False))
                    mask = cv2.inRange(img, lower_color, upper_color)
                    num_color_pixels = np.count_nonzero(mask)

                    # Calculate the total number of pixels
                    num_pixels = img.shape[0] * img.shape[1]

                    # Calculate the percentage of blue pixels in the image
                    selected_color_percentage = (num_color_pixels / num_pixels) * 100

                    #검정일경우 추가루틴 검정범위이나 다른색에 가까운 이미지
                    if category_folder == '검정':

                        # Create a mask for black and near-black pixels
                        # Define black as those pixels where all three of the R, G, B values are below 50,
                        # and the difference between the maximum and minimum of the three values is below a certain threshold
                        threshold_diff = 10
                        black_pixels = np.where(  #검정색 추출 
                            (img[:, :, 0] <= upper_color[0]) &
                            (img[:, :, 1] <= upper_color[1]) &
                            (img[:, :, 2] <= upper_color[2]) &

                            #검정색이나 회색아닌것 추출 
                            (np.max(img, axis=2) - np.min(img, axis=2) < threshold_diff))

                        # Calculate and print the percentage of black pixels
                        percentage = len(black_pixels[0]) * 100.0 / num_pixels
                        #print(f"Percentage of black pixels: {percentage:.2f}%")

                        # If the percentage is greater than 10%, do something
                        if percentage >= threshold:

                            #print("More than 10% of the image consists of black pixels.")
                            pass
                        else:
                            #print('검정범위이나 다른색에 가까운 이미지 = ',os.path.join(src_dir, file))
                            #shutil.copy(os.path.join(src_dir, file), '검정외')
                            continue

                        other_pixels = np.where(  #검정색외 흰색아닌거 추출 
                            ((img[:, :, 0] > 60) |
                             (img[:, :, 1] > 60) |
                             (img[:, :, 2] > 54)) &

                            #흰색 아닌거 추출
                            ~(((img[:, :, 0] >= 230) & (img[:, :, 0] <= 255)) &
                              ((img[:, :, 1] >= 230) & (img[:, :, 1] <= 255)) &
                              ((img[:, :, 2] >= 230) & (img[:, :, 2] <= 255))) &

                            #회색아닌거 추출 
                            ~((img[:, :, 0] == img[:, :, 1]) & (img[:, :, 0] == img[:, :, 2]))

                        )

                        other_percentage = len(other_pixels[0]) * 100.0 / num_pixels
                        #print(other_percentage)

                        #1%이상 다른 색상 
                        other_color_threshold = 1

                        # If the percentage is greater than 1%, do something
                        if other_percentage >= other_color_threshold:
                            continue

                    #청록일 경우 G:B 비율 1:1로 
                    if category_folder == '청록':
                        # Mask for pixels within the desired color range
                        color_range_mask = cv2.inRange(img, lower_color, upper_color)

                        # Mask for pixels where B is greater than or equal to G
                        b_ge_g_mask = (img[:, :, 2] >= img[:, :, 1]).astype(np.uint8)

                        # Mask for pixels where the difference between B and G is within 20
                        b_g_diff_mask = (np.abs(img[:, :, 2] - img[:, :, 1]) <= 20).astype(np.uint8)

                        # Combine the masks
                        gb_condition_mask = cv2.bitwise_and(b_ge_g_mask, b_g_diff_mask)

                        # Final mask: pixels that satisfy all conditions
                        final_mask = cv2.bitwise_and(color_range_mask, gb_condition_mask)

                        num_color_pixels = np.count_nonzero(final_mask)
                        num_pixels = img.shape[0] * img.shape[1]
                        selected_color_percentage = (num_color_pixels / num_pixels) * 100

                    # Copy the file to the destination directory if it is blue
                    if selected_color_percentage >= threshold:

                        if not os.path.exists(dst_dir):
                            os.makedirs(dst_dir)

                        shutil.copy(os.path.join(src_dir, vienna_code_folder, image_path), dst_dir)

                        COUNT_VIENNA += 1
                        #escape if count match max 
                        if COUNT_VIENNA >= each_vienna_max:
                            print(vienna_code_folder, COUNT_VIENNA)
                            break



                else:
                    #all_copy_count += 1
                    #print('no = ',all_copy_count)
                    pass


for i in lower_color_dic.keys():
    all_copy_with_sub_folder(match_category=i)

빨강
240907_TOTAL 1
270916_TOTAL 1
260406_TOTAL 1
020120_TOTAL 1
110304_TOTAL 1
040102_TOTAL 1
270501_TOTAL 1
020301_TOTAL 1
250307_TOTAL 1
270315_TOTAL 1
050320_TOTAL 1
270525_TOTAL 1
050101_TOTAL 1
030925_TOTAL 1
240715_TOTAL 1
260212_TOTAL 1
261113_TOTAL 1
010115_TOTAL 1
260113_TOTAL 1
290101_TOTAL 1
261111_TOTAL 1
260111_TOTAL 1
260210_TOTAL 1
020919_TOTAL 1
241322_TOTAL 1
261108_TOTAL 1
031324_TOTAL 1
260524_TOTAL 1
050306_TOTAL 1
270503_TOTAL 1
260705_TOTAL 1
270914_TOTAL 1
260404_TOTAL 1
270112_TOTAL 1
240905_TOTAL 1
240103_TOTAL 1
270507_TOTAL 1
061909_TOTAL 1
010316_TOTAL 1


KeyboardInterrupt: 